In [ ]:
from urllib.parse import urlparse, urldefrag, urljoin
from urllib.request import urlopen
from bs4 import BeautifulSoup
from queue import Queue
import time as time

pip install url_normalize

from url_normalize import url_normalize

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import files

In [ ]:
def download_from_the_internet(url):
    '''Скачивает сраницу с интернета

    Параметры:
        url (str) - ссылка на страницу для скачивания

    Возвращает:
        str - html-страница в виде строки, None в случае неудачи
    '''
    try:
        return urlopen(url).read().decode('utf-8')
    except KeyboardInterrupt:
        raise
    except:
        return None

    
def extract_links_from_html(url, html):
    '''Парсит ссылки на странице

    Принимает:
        url (str) - исходный урл страницы
        html (str) - содержание html-страницы

    Возвращает:
        list - список ссылок, находящихся на странице
    '''
    parser = BeautifulSoup(html)
    # Формируем ссылки на те страницы, на которые ссылается документ
    return [urljoin(url, link.get('href')) for link in parser.findAll('a')]


def extract_text_info_from_html(html):
    '''Парсит текстовую информацию на странице

    Принимает:
         html (str) - содержание html-страницы

    Возвращает:
        dict - текстовая часть страницы по ключу text,
               название по ключу title
    '''
    soup = BeautifulSoup(html, features="html.parser")
    for script in soup(["script", "style"]):
        script.extract()
    
    # Объединяем строки текста
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    # Находим название на странице
    title = soup.find('title').string
    
    return {'text': text, 'title': title}

In [1]:
def load_web_pages(seed, max_downloads, check_link, filtration_function, adjust_function):
    '''Обходит web-страницы в ширину и загружает информацию о них.
    
    Принимает:
        seed (str) -- страница, с которой начинать обход.
        max_downloads (int) -- максимальное число загруженных страниц.
        filtration_function (str, str -> bool) -- функция, указывающая, 
            стоит ли загружать страницу. Пример: is_wiki_article.
        check_link (str) -- ссылка, по которой работает ф-ия фильтрации
        adjust_function (dict -> dict) - функция, которая фиксит то, что
                                        вытянули со странички
            
    Возвращает:
        pages_json (list) - список словарей с информацией о страницах.
    '''
    
    # Создаём список со страницами
    pages_json = []
    
    # Создаём очередь для обхода в ширину
    q = Queue()
    q.put(seed)

    already_visited = set()
    n_downloads = 0
    time_start = time.time()

    while not q.empty():
        # Нормализуем урл
        main_url = url_normalize(q.get())
        if main_url in already_visited:
            continue
        already_visited.add(main_url)
        html = download_from_the_internet(main_url)

        # Извлекаем ссылки из страницы
        if not(html is None):
            children_links = extract_links_from_html(main_url, html)
            time.sleep(1)

            # Извлекаем текст страницы
            text_info = extract_text_info_from_html(html)
            text_info['url'] = main_url

            # Добавляем запись в таблицу
            pages_json.append(adjust_function(text_info))

            n_downloads += 1
            if n_downloads > max_downloads:
                break

            # Добавляем ещё не посещённые ссылки в очередь
            for child in children_links:
                if url_normalize(child) not in already_visited \
                and filtration_function(check_link, child):
                    q.put(child)
                
    return pages_json


def is_article(link, url):
    '''Проверяет, является ли ссылка нужной страницей'''
    ''' На вход:    link - какое-то ключевое слово, по которому првоерка
                    url - ссылка, которую проверяем'''
    if (link == url):
        return False
    if (link in url):
        return True
    return False


def check_integer(string):
  return any(map(str.isdigit, string))

def rework_info(text_info):
    '''
    Фиксит то, что вытянулось в text_info
    '''
    text_parts = text_info['text'].split('\n')
    variety_info = dict() 
    variety_info['url'] = text_info['url']
    variety_info['name'] = text_parts[56].split('/')[0]
    variety_info['unique_text'] = text_parts[64]
    return variety_info

In [ ]:
# константы
URL_SEED = "https://altsemena.org/catalog/semena/ovoshchi-i-zelennye-kultury/tomat/" # ссылка на каталог со страницами
PAGES_SITE = 5 # кол-во страниц в общем каталоге
MAX_PAGES = 170 # кол-во страниц в итоговом датасете

In [ ]:
for i in range(PAGES_SITE):
  seed = ''
  if i == 0:
    seed = URL_SEED
  else:
    seed = URL_SEED + '?PAGEN_1' + str(i + 1)

  json_data = load_web_pages(seed, MAX_PAGES, URL_SEED, is_article, rework_info)
  new_df = pd.DataFrame(json_data)

In [ ]:
new_df.head(10)

In [ ]:
new_df.to_csv('seeds_df.csv')
files.download('seeds_df.csv')